In [3]:
import cv2
import numpy as np
import pytesseract
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

# Load pre-trained CNN model (e.g., VGG16) for document classification
base_model = keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

# Add custom classification layers
model = keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')  # 5 document classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# OCR Function
def extract_text(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

# Anomaly Detection (Forgery Check)
def detect_forgery(image_path):
    image = cv2.imread(image_path, 0)
    edges = cv2.Canny(image, 100, 200)
    avg_edge_intensity = np.mean(edges)
    return avg_edge_intensity < 50  # Flag as forged if edge intensity is low

# Document Verification
class DocumentVerifier:
    def __init__(self):
        self.classifier = RandomForestClassifier(n_estimators=100)
    
    def train(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        self.classifier.fit(X_train, y_train)
    
    def verify(self, image_path, expected_text):
        doc_class = np.argmax(model.predict(image_path))
        extracted_text = extract_text(image_path)
        is_forged = detect_forgery(image_path)
        text_match = expected_text.lower() in extracted_text.lower()
        return {'doc_class': doc_class, 'text_match': text_match, 'is_forged': is_forged}

with open('fraud_model.pkl', 'wb') as file:
    pickle.dump(model, file)